Modeling in Julia -- Materials
==============================

**Date:** [2023-06-24 Sat 22:07]



## Heat Conduction Model



In [1]:
using ModelingToolkitStandardLibrary.Thermal, ModelingToolkit, OrdinaryDiffEq, Plots

In [1]:
@parameters t

C1 = 15
C2 = 15
@named mass1 = HeatCapacitor(C = C1, T_start = 373.15)
@named mass2 = HeatCapacitor(C = C2, T_start = 273.15)
@named conduction = ThermalConductor(G = 10)
@named Tsensor1 = TemperatureSensor()
@named Tsensor2 = TemperatureSensor()

connections = [
    connect(mass1.port, conduction.port_a),
    connect(conduction.port_b, mass2.port),
    connect(mass1.port, Tsensor1.port),
    connect(mass2.port, Tsensor2.port),
]

@named model = ODESystem(connections, t,
                         systems = [mass1, mass2, conduction, Tsensor1, Tsensor2])
sys = structural_simplify(model)
prob = ODEProblem(sys, Pair[], (0, 5.0))
sol = solve(prob, Tsit5())

T_final_K = sol[(mass1.T * C1 + mass2.T * C2) / (C1 + C2)]

plot(title = "Thermal Conduction Demonstration")
plot!(sol, idx = [mass1.T, mass2.T], labels = ["Mass 1 Temperature" "Mass 2 Temperature"])
plot!(sol.t, T_final_K, label = "Steady-State Temperature")

#+attr_org: :width 1146

## Rabbits and wolves revisited



### Single trajectory



In [1]:
using ModelingToolkit, DifferentialEquations, Plots

# Define our state variables: state(t) = initial condition
@variables ⌚ 🐰(⌚)=1 🐺(⌚)=1 🐰🐺(⌚)=2

# Define our parameters
@parameters α=1.5 β=1.0 γ=3.0 δ=1.0

# Define our differential: takes the derivative with respect to `t`
D = Differential(⌚)

# Define the differential equations
eqs = [D(🐰) ~ α * 🐰 - β * 🐰 * 🐺
       D(🐺) ~ -γ * 🐺 + δ * 🐰 * 🐺
       🐰🐺 ~ 🐰 + 🐺]

# Bring these pieces together into an ODESystem with independent variable t
@named sys = ODESystem(eqs, ⌚)

# Symbolically Simplify the System
simpsys = structural_simplify(sys)

# Convert from a symbolic to a numerical problem to simulate
tspan = (0.0, 10.0)
prob = ODEProblem(simpsys, [], tspan)

# Solve the ODE
sol = solve(prob, reltol=1e-8)

# Plot the solution
# p1 = plot(sol, title = "Rabbits vs Wolves")
# p2 = plot(sol, idxs = z, title = "Total Animals")

# plot(p1, p2, layout = (2, 1))

In [1]:
plot([sol.u[i][1] for i in 1:length(sol.u)], 
[sol.u[i][2] for i in 1:length(sol.u)], title = "Phase Diagram",
     xlabel="Rabbits", ylabel="Wolves")

#+attr_org: :width 1140

### Ensemble solution



In [1]:
# Function to set different initial conditions for each problem in the ensemble
initial_conditions = [[x,y] for x in LinRange(3,6,1000) for y in LinRange(3,6,100)];

f8c6fdf8-acb8-4e60-a5ea-1789b28307ea

In [1]:
function prob_func(prob, i, repeat)
    remake(prob, u0 = initial_conditions[i])
end
ensemble_prob = EnsembleProblem(prob, prob_func = prob_func)

a92a9a3c-0f47-4ca7-91d7-ae1c55abcd50

In [1]:
@time sim = solve(ensemble_prob, Tsit5(), EnsembleThreads(), trajectories = 100000, reltol=1e-8);

26.808094 seconds (256.23 M allocations: 19.276 GiB, 20.28% gc time, 31.12% compilation time)

![img](./.ob-jupyter/21a279939ce6593bfdc2d69aaae03aab4adaca64.svg)



### GPU accelerated ensemble solution



In [1]:
using CUDA
CUDA.devices()

CUDA.DeviceIterator() for 2 devices:
0. NVIDIA GeForce GTX 1080 Ti
1. NVIDIA GeForce GTX 980

In [1]:
# Select a specific GPU device by its index (indexing starts from 0)
CUDA.device!(0) # Sets the second GPU as the active device

CuDevice(0): NVIDIA GeForce GTX 1080 Ti

In [1]:
initial_conditions = [[x,y] for x in LinRange(3,6,1000) for y in LinRange(3,6,100)];
function prob_func(prob, i, repeat)
    remake(prob, u0 = initial_conditions[i])
end
ensemble_prob = EnsembleProblem(prob, prob_func = prob_func, safetycopy = false)

EnsembleProblem with problem ODEProblem

In [1]:
using DiffEqGPU
@time sim_gpu = solve(ensemble_prob, Tsit5(), 
                      EnsembleGPUArray(CUDA.CUDABackend()), trajectories = 100000, reltol=1e-8, saveat = 1.0f0);

17.616470 seconds (3.63 M allocations: 144.059 GiB, 13.51% gc time)

In [1]:
phase_plot = plot()
for i in 1:length(sim_gpu)
    sol = sim_gpu[i]
    # Extract relevant data for phase plot
    rabbits = [sol.u[j][1] for j in 1:length(sol.u)]
    wolves = [sol.u[j][2] for j in 1:length(sol.u)]

    # Plot the phase diagram for the current trajectory
    plot!(phase_plot, rabbits, wolves, label=false, xlabel="Rabbits", ylabel="Wolves", title="Phase Diagram")
end

display(phase_plot)

## Stochastic Differential Equations



### Single path



In [1]:
using ModelingToolkit, StochasticDiffEq
@parameters sigma
@variables t x(t)
D = Differential(t)
eqs = [D(x) ~ 0]
noiseeqs = [1]
@named de = SDESystem(eqs, noiseeqs, t, [x], []);

u0map = [x => 0]

prob = SDEProblem(de, u0map, (0,10.))
sol = solve(prob, SOSRI())
plot(sol, title = "A sample path of Wiener process",
     xlabel = "Time", ylabel = "Displacement")

#+attr_org: :width 1142

### Ensemble path and its statistics



In [1]:
function prob_func_sde(prob, i, repeat)
    remake(prob)
end
ensemble_prob_sde = EnsembleProblem(prob, prob_func = prob_func_sde)

EnsembleProblem with problem SDEProblem

In [1]:
@time sim_sde = solve(ensemble_prob_sde, SOSRI(), EnsembleThreads(), trajectories = 100000, reltol=1e-8);

3.041548 seconds (35.50 M allocations: 3.153 GiB, 22.70% gc time)

In [1]:
displacement_at_i = [sim_sde[j].u[3][1] for j in 1:length(sim_sde)]

-   Some statistics



In [1]:
using Statistics
num_time_steps = length(sim_sde[1].t)
variance_t = zeros(num_time_steps)

for i in 1:num_time_steps
    displacement_at_i = [sim_sde[j].u[i][1] for j in 1:length(sim_sde)]
    variance_t[i] = var(displacement_at_i)
end

In [1]:
plot(sim_sde[1].t, variance_t, xlabel= "Time", ylabel = "var{x}", title = "Variance of displacement vs time")

#+attr_org: :width 1144